In [1]:
%pylab inline
from glob import glob
from numpy import load
import pandas as pd
import numpy as np
import copy

from time import time
import pickle as pkl
import os
import cv2
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os
path = os.getcwd()
_uname = path.split('/')[2]
poverty_dir=f'/home/{_uname}/public/cs255-sp22-a00-public/poverty/'
image_dir=poverty_dir+'anon_images/'
image_dir

'/home/ans037/public/cs255-sp22-a00-public/poverty/anon_images/'

In [3]:
train_table=f'/home/{_uname}/public/Datasets_public/Final_Project_Data/train.csv'
df=pd.read_csv(train_table,index_col=0)
df.index=df['filename']

In [4]:
def getImage(image):
    M = np.load(image)
    l = []
    for i in range(8):
        l.append(cv2.resize(M['x'][ i , :, :], (20, 20)))
    return np.array(l)

In [5]:
df['Image'] = df.apply(lambda x: getImage(image_dir + '/'+ x['filename']), axis=1)

In [6]:
df1 = df.copy(deep=True)

In [7]:
def calculate_indices(df, train=True):
    if train:
        df = df.drop(['filename', 'wealthpooled'], axis=1)
    else:
        df = df.drop(['filename'], axis=1)
    
    df = df.reset_index(drop=True)
    band_names = ['Red', 'Green', 'Blue', 'NIR', 'SWIR1', 'SWIR2', 'TEMP1', 'NL']
    
    for i in range(len(band_names)):
        df[band_names[i]] = df.apply(lambda x: x['Image'][i, :, :], axis=1)

    # Calculate the bare soil indices
    df['MBI'] = (df['SWIR1'] - df['SWIR1'] - df['NIR']) / (df['SWIR1'] + df['SWIR2'] + df['NIR']) + 0.5
    df['BSI'] = (df['SWIR1'] + df['Red'] - df['NIR'] - df['Blue']) / (df['SWIR1'] + df['Red'] + df['NIR'] + df['Blue'])
    #df['BSI1'] = (df['SWIR1'] + df['Red'] - df['NIR'] - df['Blue']) / (df['SWIR1'] + df['Red'] + df['NIR'] + df['Blue'])
    #df['BSI3'] = ((df['SWIR1'] + df['Red'] - df['NIR'] - df['Blue']) / (df['SWIR1'] + df['Red'] + df['NIR'] + df['Blue'])) * 100 + 100
    df['NDSI1'] = (df['SWIR1'] - df['NIR']) / (df['SWIR1'] + df['NIR'])
    df['NDSI2'] = (df['SWIR1'] - df['Green']) / (df['SWIR1'] + df['Green'])
    df['BI'] = df['Red'] + df['SWIR1'] - df['NIR']
    df['DBSI'] = (df['SWIR1'] - df['Green']) / (df['SWIR1'] + df['Green']) - (df['NIR'] - df['Red']) / (df['NIR'] + df['Red'])

    df['BAEI'] = (df['Red'] + 0.3) / (df['Green'] + df['SWIR1'])
    df['BUI'] = (df['SWIR1'] - df['NIR']) / (df['SWIR1'] + df['NIR']) - (df['NIR'] - df['Red']) / (df['NIR'] + df['Red'])
    df['NBI'] = (df['Red'] - df['SWIR1']) / df['NIR']
    df['BRBA'] = df['Red'] / df['SWIR1']
    df['IBI'] = (2 * df['SWIR1'] / (df['SWIR1'] + df['NIR'])) - ((df['NIR'] / (df['NIR'] - df['Red']) + df['Green'] / (df['Green'] + df['SWIR1'])) / 2) * (df['SWIR1'] / (df['SWIR1'] + df['NIR'])) + (df['NIR'] / (df['NIR'] - df['Red']) + df['Green'] / (df['Green'] + df['SWIR1']))
    df['NDCCI'] = (df['NIR'] - df['Green']) / (df['NIR'] + df['Green'])

    # Vegetation indices
    df['NDVI'] = (df['NIR'] - df['Red']) / (df['NIR'] + df['Red'])
    df['SAVI'] = (df['NIR'] - df['Red']) / (df['NIR'] + df['Red'] + 0.5) * (1 + 0.5)
    df['NDMI'] = (df['NIR'] - df['SWIR1']) / (df['NIR'] + df['SWIR1'])

    # Built-up area indices
    df['DBI'] = (df['Blue'] - df['TEMP1']) / (df['Blue'] + df['TEMP1']) - (df['NIR'] - df['Red']) / (df['NIR'] + df['Red'])
    df['NBAI'] = ((df['SWIR2'] - df['SWIR1']) / df['Green']) / ((df['SWIR2'] + df['SWIR1']) / df['Green'])
    df['NDBI'] = (df['SWIR1'] - df['NIR']) / (df['SWIR1'] + df['NIR'])

    indices = ['MBI', 'BSI', 'NDSI1', 'NDSI2', 'BI', 'DBSI', 'BAEI', 'BUI', 'NBI', 'BRBA', 'IBI', 'NDCCI', 'NDVI', 'SAVI', 'NDMI', 'DBI', 'NBAI', 'NDBI']

    for feature in list(indices + band_names):
        df[f'{feature}_mean'] = df[f'{feature}'].apply(lambda x: np.mean(x))
        df[f'{feature}_std'] = df[f'{feature}'].apply(lambda x: np.std(x))
        df[f'{feature}_median'] = df[f'{feature}'].apply(lambda x: np.median(x))

    df = df.drop(band_names, axis=1)
    df = df.drop(indices, axis=1)
    df = df.drop('Image', axis=1)
    
    return df

In [8]:
df1 = calculate_indices(df1, train=True)

/tmp/ipykernel_376/3243455848.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{feature}_median'] = df[f'{feature}'].apply(lambda x: np.median(x))


In [9]:
df1.columns

Index(['country', 'urban', 'label', 'nl_mean', 'MBI_mean', 'MBI_std',
       'MBI_median', 'BSI_mean', 'BSI_std', 'BSI_median', 'NDSI1_mean',
       'NDSI1_std', 'NDSI1_median', 'NDSI2_mean', 'NDSI2_std', 'NDSI2_median',
       'BI_mean', 'BI_std', 'BI_median', 'DBSI_mean', 'DBSI_std',
       'DBSI_median', 'BAEI_mean', 'BAEI_std', 'BAEI_median', 'BUI_mean',
       'BUI_std', 'BUI_median', 'NBI_mean', 'NBI_std', 'NBI_median',
       'BRBA_mean', 'BRBA_std', 'BRBA_median', 'IBI_mean', 'IBI_std',
       'IBI_median', 'NDCCI_mean', 'NDCCI_std', 'NDCCI_median', 'NDVI_mean',
       'NDVI_std', 'NDVI_median', 'SAVI_mean', 'SAVI_std', 'SAVI_median',
       'NDMI_mean', 'NDMI_std', 'NDMI_median', 'DBI_mean', 'DBI_std',
       'DBI_median', 'NBAI_mean', 'NBAI_std', 'NBAI_median', 'NDBI_mean',
       'NDBI_std', 'NDBI_median', 'Red_mean', 'Red_std', 'Red_median',
       'Green_mean', 'Green_std', 'Green_median', 'Blue_mean', 'Blue_std',
       'Blue_median', 'NIR_mean', 'NIR_std', 'NIR_median', 

## Training Random Forest

In [10]:
X = df1.drop('label', axis=1)
y = df1['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def train_random_forest(X_train, y_train):
    rf = RandomForestClassifier(n_estimators=200, max_depth=10, bootstrap=True, oob_score=True)  # Customize the Random Forest hyperparameters as per your requirements
    rf.fit(X_train, y_train)
    return rf

def evaluate_model(model, X, y):
    y_pred = model.predict(X)
    accuracy = accuracy_score(y, y_pred)
    return accuracy

In [12]:
rf_model = train_random_forest(X_train, y_train)

rf_train_accuracy = evaluate_model(rf_model, X_train, y_train)
    
rf_test_accuracy = evaluate_model(rf_model, X_test, y_test)

In [13]:
rf_train_accuracy, rf_test_accuracy

(0.9058693244739756, 0.7657218777679362)